# Lab 4

In [1]:
import numpy as np
import pandas as pd
import requests
from lxml import html

Go [here](https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29) and scrape the GDP by country determined by the UN, save in DataFrame.
* Some of the country rows are actually footnote links, so filter out with `[not(contains(text(),"["))]`
* First row is the total, so truncate it
* Convert string to int for GDP

In [104]:
pageContent = requests.get('https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29')
tree = html.fromstring(pageContent.content)
countries = tree.xpath('//table/tbody/tr/td[3]/table/tbody/tr/td[2]//a[1][not(contains(text(),"["))]/text()')[1:]
gdp = [int(string.replace(',','')) for string in tree.xpath('//table/tbody/tr/td[3]/table/tbody/tr/td[3]//span/text()')][1:]
df1 = pd.DataFrame(data={'Country':countries,'GDP':gdp})
df1

,Country,GDP
0,United States,19485394
1,China,12234781
2,Japan,4872415
3,Germany,3693204
4,United Kingdom,2631228
...,...,...
207,Marshall Islands,209
208,Nauru,182
209,Kiribati,180
210,Montserrat,63


Go [here](https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population) and scrape the population of each country, save in DataFrame
* Some of the country rows are actually footnote links, so filter out with `[not(contains(text(),"["))]`
* Don't include territories, which have "-" or "—" in the rank column, filter out with `[not(td[1]/text()="–" or td[1]/text()="—")]`
* Convert string to int for population

In [103]:
pageContent = requests.get('https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population')
tree = html.fromstring(pageContent.content)
countries = tree.xpath('//table/tbody/tr[not(td[1]/text()="–" or td[1]/text()="—")]/td[2][a[1]/text() or span/a[1]/text()]//a[1][not(contains(text(),"["))]/text()')
population = [int(string.replace(',','')) for string in tree.xpath('//table/tbody/tr[not(td[1]/text()="–" or td[1]/text()="—")]/td[3]/text()')]
df2 = pd.DataFrame({'Country':countries,'Population':population})
df2

,Country,Population
0,China,1402494800
1,India,1361825296
2,United States,329629242
3,Indonesia,266911900
4,Pakistan,220892331
...,...,...
190,San Marino,33553
191,Palau,17900
192,Nauru,11000
193,Tuvalu,10200


Do an inner join on the 2 DataFrames

In [105]:
df3 = df1.set_index('Country').join(df2.set_index('Country'),how="inner")
df3

,GDP,Population
Country,,
United States,19485394,329629242
China,12234781,1402494800
Japan,4872415,125950000
Germany,3693204,83149300
United Kingdom,2631228,66435550
...,...,...
Palau,234,17900
Marshall Islands,209,55500
Nauru,182,11000


Determine the correlation between the GDP of a country and its population

In [106]:
df3['GDP'].corr(df3['Population'])

0.5623686142718832